<a href="https://colab.research.google.com/github/Shi-Yile/SPH6004-Assignment-1/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
# Import packages

# Data-processing packages
import pandas as pd
import numpy as np

# Plotting packages
import matplotlib.pyplot as plt
import seaborn as sn

# ML packages
import sklearn

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, StratifiedKFold, cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from sklearn.metrics import roc_auc_score, roc_curve, classification_report, confusion_matrix, recall_score, accuracy_score


import os
from google.colab import drive

In [7]:
# give permission to load data from google drive
drive.mount('/content/drive')
os.chdir('drive/My Drive/SPH6004/Assignment-1')

Mounted at /content/drive


In [8]:
# load dataset
df = pd.read_csv('data.csv')
df_origin = df.copy()
df.head()

,id,aki,gender,admission_age,race,heart_rate_min,heart_rate_max,heart_rate_mean,sbp_min,sbp_max,...,ggt_max,ld_ldh_min,ld_ldh_max,gcs_min,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable,height,weight_admit
0,36570066,3,F,79.953141,BLACK/AFRICAN AMERICAN,96.0,104.0,100.083333,103.0,126.0,...,NaN,236.0,318.0,15.0,6.0,5.0,4.0,0.0,157.0,110.0
1,39307659,0,F,78.194169,WHITE - RUSSIAN,72.0,134.0,97.263158,97.0,127.0,...,NaN,NaN,NaN,15.0,6.0,5.0,4.0,0.0,NaN,82.0
2,38743306,2,F,65.602396,WHITE,60.0,97.0,84.166667,95.0,143.0,...,NaN,NaN,NaN,15.0,6.0,5.0,4.0,0.0,NaN,62.1
3,32339865,2,F,64.906629,UNKNOWN,59.0,87.0,71.461538,113.0,150.0,...,NaN,NaN,NaN,15.0,1.0,0.0,1.0,1.0,170.0,113.1
4,35526987,2,M,57.438861,WHITE,57.0,100.0,82.387097,81.0,127.0,...,NaN,NaN,NaN,15.0,NaN,0.0,1.0,1.0,178.0,97.4


In [9]:
# basic infomration of each column in df
df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50920 entries, 0 to 50919
Data columns (total 162 columns):
 #    Column                  Non-Null Count  Dtype  
---   ------                  --------------  -----  
 0    id                      50920 non-null  int64  
 1    aki                     50920 non-null  int64  
 2    gender                  50920 non-null  object 
 3    admission_age           50920 non-null  float64
 4    race                    50920 non-null  object 
 5    heart_rate_min          50841 non-null  float64
 6    heart_rate_max          50841 non-null  float64
 7    heart_rate_mean         50841 non-null  float64
 8    sbp_min                 50823 non-null  float64
 9    sbp_max                 50823 non-null  float64
 10   sbp_mean                50823 non-null  float64
 11   dbp_min                 50823 non-null  float64
 12   dbp_max                 50823 non-null  float64
 13   dbp_mean                50823 non-null  float64
 14   mbp_min             

<ipython-input-9-1fc94d2bc7dc>:2: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(verbose = True, null_counts = True)


In [10]:
# frequencies in 'race'
df['race'].value_counts()

WHITE                                        32637
UNKNOWN                                       5579
BLACK/AFRICAN AMERICAN                        3845
OTHER                                         1745
WHITE - OTHER EUROPEAN                         918
UNABLE TO OBTAIN                               726
ASIAN                                          614
ASIAN - CHINESE                                547
HISPANIC/LATINO - PUERTO RICAN                 530
HISPANIC OR LATINO                             501
WHITE - RUSSIAN                                430
PATIENT DECLINED TO ANSWER                     371
HISPANIC/LATINO - DOMINICAN                    337
BLACK/CAPE VERDEAN                             319
BLACK/CARIBBEAN ISLAND                         282
BLACK/AFRICAN                                  194
ASIAN - SOUTH EAST ASIAN                       168
PORTUGUESE                                     161
ASIAN - ASIAN INDIAN                           121
WHITE - EASTERN EUROPEAN       

In [11]:
# re-catergorize 'race' with fewer levels, and create dummy variables
# white, black, asian, hispanic/latino, other, unknown
# df['white'] = df['race'].apply(lambda x: 1 if 'WHITE' in x else 0)
df['race_white'] = df['race'].apply(lambda x: int('WHITE' in x))
df['race_black'] = df['race'].apply(lambda x: int('BLACK' in x))
df['race_asian'] = df['race'].apply(lambda x: int('ASIAN' in x))
df['race_hispanic/latino'] = df['race'].apply(
    lambda x: int('HISPANIC' in x or 'LATINO' in x))
df['race_unknown'] = df['race'].apply(lambda x: int(
    x == 'UNKNOWN' or x == 'UNABLE TO OBTAIN' or
    x == 'PATIENT DECLINED TO ANSWER'))
df['race_other'] = df['race'].apply(lambda x: int(
    x == 'OTHER' or x == 'PORTUGUESE' or
    x == 'AMERICAN INDIAN/ALASKA NATIVE' or
    x == 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER' or
    x == 'MULTIPLE RACE/ETHNICITY' or x == 'SOUTH AMERICAN'))

# frequency in each level
print('WHITE: ', df['race_white'].sum())
print('BLACK: ', df['race_black'].sum())
print('ASIAN: ', df['race_asian'].sum())
print('HISPANIC/LATINO: ', df['race_hispanic/latino'].sum())
print('OTHER: ', df['race_other'].sum())
print('UNKNOWN: ', df['race_unknown'].sum())

WHITE:  34204
BLACK:  4640
ASIAN:  1496
HISPANIC/LATINO:  1735
OTHER:  2169
UNKNOWN:  6676


In [12]:
# recode gender as 0-1  0: male, 1: female
df['gender_'] = df['gender'].apply(lambda x:int(x == 'F'))
df['gender_'].value_counts()

0    28440
1    22480
Name: gender_, dtype: int64

In [13]:
# re-categorise AKI as a binary column  0: aki = 0, 1: aki = 1/2/3
df['aki_'] = df['aki'].apply(
    lambda x:int(x == 1 or x == 2 or x == 3))
df['aki_'].value_counts()

1    34060
0    16860
Name: aki_, dtype: int64

In [14]:
# NA detected, thus need further processing including feature selection
# calculate the proportion of NAs in each column
df_NA_prop = 1 - df.count() / len(df)

# extract two sets of indices from df
# whose correpsonding columns including NAs less than 10% and 50%, respectively
idx_NA10 = df_NA_prop[df_NA_prop <= 0.1].index
idx_NA50 = df_NA_prop[df_NA_prop <= 0.5].index
# idx_NA10

# extract two subsets of df based on previous indices
df_sub_NA10 = df[idx_NA10]
df_sub_NA50 = df[idx_NA50]
# df_sub_NA10.info()

#  drop rows with NAs
df_sub_com10 = df_sub_NA10.dropna()
df_sub_com50 = df_sub_NA50.dropna()
# df_sub_com10.info()

In [ ]:
# df_sub_com10.info()
# df_sub_NA50.info()

In [83]:
df_sub_com10['aki_'].value_counts()

1    26484
0    12147
Name: aki_, dtype: int64

In [84]:
df_sub_com50['aki_'].value_counts()

1    10569
0     2945
Name: aki_, dtype: int64

In [16]:
# create outcome vector and predictor matrix
# drop id and race first
df_10 = df_sub_com10.drop(columns = ['id', 'aki', 'race', 'gender'])
df_50 = df_sub_com50.drop(columns = ['id', 'aki', 'race', 'gender'])

In [17]:
# split dataset
# first we extract X and y

# df_10
y_df_10 = df_10['aki_']
X_df_10 = df_10.drop(columns = ['aki_'])

# df_50
y_df_50 = df_50['aki_']
X_df_50 = df_50.drop(columns = ['aki_'])

In [18]:
# standardization on X: (X - mean) / std
Std = StandardScaler(copy = False)
X_std_df_10 = pd.DataFrame(
    data = Std.fit_transform(X_df_10),
    columns = Std.get_feature_names_out(input_features = X_df_10.columns))

X_std_df_50 = pd.DataFrame(
    data = Std.fit_transform(X_df_50),
    columns = Std.get_feature_names_out(input_features = X_df_50.columns))

In [19]:
# training - test split
X_10_train, X_10_test, y_10_train, y_10_test = train_test_split(
    X_std_df_10, y_df_10, test_size = 0.3, random_state = 42,
    stratify = y_df_10, shuffle = True)

X_50_train, X_50_test, y_50_train, y_50_test = train_test_split(
    X_std_df_50, y_df_50, test_size = 0.3, random_state = 42,
    stratify = y_df_50, shuffle = True)

In [23]:
# logistic regression with CV
lr_param = {
    'C': np.linspace(0.001, 0.01, 10),
}

In [21]:
# GridSearch with CV for best C
stratifiedCV = StratifiedKFold(n_splits = 3)
LR = LogisticRegression(
    class_weight = 'balanced',
    solver = 'saga',
    max_iter = 50000)
BestLR = GridSearchCV(
    LR,
    param_grid = lr_param,
    scoring = 'f1',
    cv = stratifiedCV,
    verbose = 1,
    n_jobs = -1
)

In [26]:
# case: df_10
BestLR.fit(X_10_train, y_10_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


{'C': 0.01}

In [29]:
print(BestLR.best_params_)
print(BestLR.best_score_)

{'C': 0.01}
0.7350173613291654


In [30]:
y_10_BestLR_pred = BestLR.predict(X_10_test)

print(confusion_matrix(y_10_test, y_10_BestLR_pred))
print(pd.DataFrame(classification_report(
    y_10_test, y_10_BestLR_pred, labels = None,
    target_names = ['no aki', 'aki'], sample_weight = None,
    digits = 2, output_dict = True)).round(2))

[[2555 1089]
 [2683 5263]]
            no aki      aki  accuracy  macro avg  weighted avg
precision     0.49     0.83      0.67       0.66          0.72
recall        0.70     0.66      0.67       0.68          0.67
f1-score      0.58     0.74      0.67       0.66          0.69
support    3644.00  7946.00      0.67   11590.00      11590.00


In [53]:
# case: df_50
BestLR.fit(X_50_train, y_50_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=LogisticRegression(class_weight='balanced',
                                          max_iter=50000, solver='saga'),
             n_jobs=-1,
             param_grid={'C': array([0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009,
       0.01 ])},
             scoring='f1', verbose=1)

In [54]:
print(BestLR.best_params_)
print(BestLR.best_score_)

{'C': 0.007}
0.7582743677497265


In [57]:
y_50_BestLR_pred = BestLR.predict(X_50_test)

print(confusion_matrix(y_50_test, y_50_BestLR_pred))
print(pd.DataFrame(classification_report(
    y_50_test, y_50_BestLR_pred, labels = None,
    target_names = ['no aki', 'aki'], sample_weight = None,
    digits = 2, output_dict = True)).round(2))

[[ 621  263]
 [1036 2135]]
           no aki      aki  accuracy  macro avg  weighted avg
precision    0.37     0.89      0.68       0.63          0.78
recall       0.70     0.67      0.68       0.69          0.68
f1-score     0.49     0.77      0.68       0.63          0.71
support    884.00  3171.00      0.68    4055.00       4055.00


# Tree-based Model
## Decision Tree

In [75]:
# Tree-base models
# decision tree
dtc_param = {'max_depth': [2, 3, 4, 5, 6, 7]}

In [76]:
TreeModel = DecisionTreeClassifier(
    criterion = 'entropy',
    class_weight = 'balanced'
    )
BestDTC = GridSearchCV(
    TreeModel,
    param_grid = dtc_param,
    scoring = 'f1',
    cv = stratifiedCV
)

In [78]:
# case: df_10
BestDTC.fit(X_10_train, y_10_train)

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              criterion='entropy'),
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7]}, scoring='f1')

In [79]:
print(BestDTC.best_params_)
print(BestDTC.best_score_)

{'max_depth': 2}
0.7589189330014632


In [80]:
y_10_BestDTCpred = BestDTC.predict(X_10_test)

print(confusion_matrix(y_10_test, y_10_BestDTCpred))
print(pd.DataFrame(classification_report(
    y_10_test, y_10_BestDTCpred, labels = None,
    target_names = ['no aki', 'aki'], sample_weight = None,
    digits = 2, output_dict = True)).round(2))

[[1753 1891]
 [1954 5992]]
            no aki      aki  accuracy  macro avg  weighted avg
precision     0.47     0.76      0.67       0.62          0.67
recall        0.48     0.75      0.67       0.62          0.67
f1-score      0.48     0.76      0.67       0.62          0.67
support    3644.00  7946.00      0.67   11590.00      11590.00


In [85]:
# case: df_50
BestDTC.fit(X_50_train, y_50_train)

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              criterion='entropy'),
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7]}, scoring='f1')

In [86]:
print(BestDTC.best_params_)
print(BestDTC.best_score_)

{'max_depth': 2}
0.7544027250491357


In [87]:
y_50_BestDTCpred = BestDTC.predict(X_50_test)

print(confusion_matrix(y_50_test, y_50_BestDTCpred))
print(pd.DataFrame(classification_report(
    y_50_test, y_50_BestDTCpred, labels = None,
    target_names = ['no aki', 'aki'], sample_weight = None,
    digits = 2, output_dict = True)).round(2))

[[ 450  434]
 [ 769 2402]]
           no aki      aki  accuracy  macro avg  weighted avg
precision    0.37     0.85       0.7       0.61          0.74
recall       0.51     0.76       0.7       0.63          0.70
f1-score     0.43     0.80       0.7       0.61          0.72
support    884.00  3171.00       0.7    4055.00       4055.00


## Random Forest

In [125]:
# random forest
rf_param = {
    'n_estimators': np.arange(start = 10, stop = 100, step = 10),
    'max_depth': np.arange(start = 2, stop = 10, step = 1)
}

In [126]:
RFModel = RandomForestClassifier(
    criterion = 'entropy',
    class_weight = 'balanced'
    )
BestRF = GridSearchCV(
    RFModel,
    param_grid = rf_param,
    scoring = 'f1',
    cv = stratifiedCV,
    verbose = 1,
    n_jobs = -1
)

In [127]:
BestRF.fit(X_10_train, y_10_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(class_weight='balanced',
                                              criterion='entropy'),
             n_jobs=-1,
             param_grid={'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9]),
                         'n_estimators': array([10, 20, 30, 40, 50, 60, 70, 80, 90])},
             scoring='f1', verbose=1)

In [128]:
print(BestRF.best_params_)
print(BestRF.best_score_)

{'max_depth': 9, 'n_estimators': 60}
0.7691816442842349


In [129]:
y_10_BestRFpred = BestRF.predict(X_10_test)

print(confusion_matrix(y_10_test, y_10_BestRFpred))
print(pd.DataFrame(classification_report(
    y_10_test, y_10_BestRFpred, labels = None,
    target_names = ['no aki', 'aki'], sample_weight = None,
    digits = 2, output_dict = True)).round(2))

[[2343 1301]
 [2275 5671]]
            no aki      aki  accuracy  macro avg  weighted avg
precision     0.51     0.81      0.69       0.66          0.72
recall        0.64     0.71      0.69       0.68          0.69
f1-score      0.57     0.76      0.69       0.66          0.70
support    3644.00  7946.00      0.69   11590.00      11590.00


In [130]:
# case: df_50
BestRF.fit(X_50_train, y_50_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(class_weight='balanced',
                                              criterion='entropy'),
             n_jobs=-1,
             param_grid={'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9]),
                         'n_estimators': array([10, 20, 30, 40, 50, 60, 70, 80, 90])},
             scoring='f1', verbose=1)

In [131]:
print(BestRF.best_params_)
print(BestRF.best_score_)

{'max_depth': 9, 'n_estimators': 90}
0.8457272099213906


In [132]:
y_50_BestRFpred = BestRF.predict(X_50_test)

print(confusion_matrix(y_50_test, y_50_BestRFpred))
print(pd.DataFrame(classification_report(
    y_50_test, y_50_BestRFpred, labels = None,
    target_names = ['no aki', 'aki'], sample_weight = None,
    digits = 2, output_dict = True)).round(2))

[[ 447  437]
 [ 557 2614]]
           no aki      aki  accuracy  macro avg  weighted avg
precision    0.45     0.86      0.75       0.65          0.77
recall       0.51     0.82      0.75       0.67          0.75
f1-score     0.47     0.84      0.75       0.66          0.76
support    884.00  3171.00      0.75    4055.00       4055.00


## AdaBoost

In [96]:
# adaboost
ada_param = {
    'n_estimators': [100, 200, 500],
    'learning_rate': np.arange(start = 0.05, stop = 0.25, step = 0.05)
}

In [97]:
AdaBoostModel = AdaBoostClassifier()
BestAda = GridSearchCV(
    AdaBoostModel,
    param_grid = ada_param,
    scoring = 'f1',
    cv = stratifiedCV,
    verbose = 1,
    n_jobs = -1
)

In [98]:
# case: df_10
BestAda.fit(X_10_train, y_10_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=AdaBoostClassifier(), n_jobs=-1,
             param_grid={'learning_rate': array([0.05, 0.1 , 0.15, 0.2 ]),
                         'n_estimators': [100, 200, 500]},
             scoring='f1', verbose=1)

In [99]:
print(BestAda.best_params_)
print(BestAda.best_score_)

{'learning_rate': 0.1, 'n_estimators': 200}
0.8261643708840841


In [100]:
y_10_BestAdapred = BestAda.predict(X_10_test)

print(confusion_matrix(y_10_test, y_10_BestAdapred))
print(pd.DataFrame(classification_report(
    y_10_test, y_10_BestAdapred, labels = None,
    target_names = ['no aki', 'aki'], sample_weight = None,
    digits = 2, output_dict = True)).round(2))

[[1143 2501]
 [ 600 7346]]
            no aki      aki  accuracy  macro avg  weighted avg
precision     0.66     0.75      0.73       0.70          0.72
recall        0.31     0.92      0.73       0.62          0.73
f1-score      0.42     0.83      0.73       0.63          0.70
support    3644.00  7946.00      0.73   11590.00      11590.00


In [101]:
# case: df_50
BestAda.fit(X_50_train, y_50_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=AdaBoostClassifier(), n_jobs=-1,
             param_grid={'learning_rate': array([0.05, 0.1 , 0.15, 0.2 ]),
                         'n_estimators': [100, 200, 500]},
             scoring='f1', verbose=1)

In [103]:
print(BestAda.best_params_)
print(BestAda.best_score_)

{'learning_rate': 0.2, 'n_estimators': 200}
0.8839393641720394


In [104]:
y_50_BestAdapred = BestAda.predict(X_50_test)

print(confusion_matrix(y_50_test, y_50_BestAdapred))
print(pd.DataFrame(classification_report(
    y_50_test, y_50_BestAdapred, labels = None,
    target_names = ['no aki', 'aki'], sample_weight = None,
    digits = 2, output_dict = True)).round(2))

[[ 152  732]
 [  80 3091]]
           no aki      aki  accuracy  macro avg  weighted avg
precision    0.66     0.81       0.8       0.73          0.78
recall       0.17     0.97       0.8       0.57          0.80
f1-score     0.27     0.88       0.8       0.58          0.75
support    884.00  3171.00       0.8    4055.00       4055.00


## XGBoost

In [58]:
# xgboost
xg_param = {
    'n_estimators': [100, 200, 500],
    'max_depth': np.arange(start = 2, stop = 5, step = 1),
    'learning_rate': np.arange(start = 0.01, stop = 0.1, step = 0.01)
}

In [60]:
XGBoostModel = XGBClassifier(importance_type = 'weight')
BestXGBoost = GridSearchCV(
    XGBoostModel,
    param_grid = xg_param,
    scoring = 'f1',
    cv = stratifiedCV,
    verbose = 1,
    n_jobs = -1
)

In [61]:
# case: df_10
BestXGBoost.fit(X_10_train, y_10_train)

Fitting 3 folds for each of 81 candidates, totalling 243 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]),
                         'max_depth': array([2, 3, 4]),
                         'n_estimators': [100, 200, 500]},
             scoring='f1', verbose=1)

In [62]:
print(BestXGBoost.best_params_)
print(BestXGBoost.best_score_)

{'learning_rate': 0.02, 'max_depth': 4, 'n_estimators': 200}
0.8272823655868796


In [63]:
y_10_BestXGBpred = BestXGBoost.predict(X_10_test)

print(confusion_matrix(y_10_test, y_10_BestXGBpred))
print(pd.DataFrame(classification_report(
    y_10_test, y_10_BestXGBpred, labels = None,
    target_names = ['no aki', 'aki'], sample_weight = None,
    digits = 2, output_dict = True)).round(2))

[[1181 2463]
 [ 602 7344]]
            no aki      aki  accuracy  macro avg  weighted avg
precision     0.66     0.75      0.74       0.71          0.72
recall        0.32     0.92      0.74       0.62          0.74
f1-score      0.44     0.83      0.74       0.63          0.70
support    3644.00  7946.00      0.74   11590.00      11590.00


In [64]:
# case: df_50
BestXGBoost.fit(X_50_train, y_50_train)

Fitting 3 folds for each of 81 candidates, totalling 243 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]),
                         'max_depth': array([2, 3, 4]),
                         'n_estimators': [100, 200, 500]},
             scoring='f1', verbose=1)

In [65]:
print(BestXGBoost.best_params_)
print(BestXGBoost.best_score_)

{'learning_rate': 0.060000000000000005, 'max_depth': 2, 'n_estimators': 200}
0.8839386288153473


In [68]:
y_50_BestXGBpred = BestXGBoost.predict(X_50_test)

print(confusion_matrix(y_50_test, y_50_BestXGBpred))
print(pd.DataFrame(classification_report(
    y_50_test, y_50_BestXGBpred, labels = None,
    target_names = ['no aki', 'aki'], sample_weight = None,
    digits = 2, output_dict = True)).round(2))

[[ 135  749]
 [  65 3106]]
           no aki      aki  accuracy  macro avg  weighted avg
precision    0.68     0.81       0.8       0.74          0.78
recall       0.15     0.98       0.8       0.57          0.80
f1-score     0.25     0.88       0.8       0.57          0.75
support    884.00  3171.00       0.8    4055.00       4055.00


# SVM

In [159]:
# SVM
svm_param = {
    'C': [1, 10, 100] # np.arange(start = 1, stop = 1, step = 0.1),
    # 'kernel': ['linear', 'rbf']
    }

In [160]:
SVCModel = SVC(class_weight = 'balanced', kernel = 'rbf')
BestSVC = GridSearchCV(
    SVCModel,
    param_grid = svm_param,
    scoring = 'f1',
    cv = stratifiedCV,
    verbose = 1,
    n_jobs = -1
)

In [142]:
BestSVC.fit(X_10_train, y_10_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


KeyboardInterrupt: 

In [39]:
print(BestSVC.best_params_)
print(BestSVC.best_score_)

{'C': 0.2}
0.7591954227042549


In [42]:
y_10_BestSVCpred = BestSVC.predict(X_10_test)

print(confusion_matrix(y_10_test, y_10_BestSVCpred))
print(pd.DataFrame(classification_report(
    y_10_test, y_10_BestSVCpred, labels = None,
    target_names = ['no aki', 'aki'], sample_weight = None,
    digits = 2, output_dict = True)).round(2))

[[2442 1202]
 [2391 5555]]
            no aki      aki  accuracy  macro avg  weighted avg
precision     0.51     0.82      0.69       0.66          0.72
recall        0.67     0.70      0.69       0.68          0.69
f1-score      0.58     0.76      0.69       0.67          0.70
support    3644.00  7946.00      0.69   11590.00      11590.00


In [161]:
BestSVC.fit(X_50_train, y_50_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=SVC(class_weight='balanced'), n_jobs=-1,
             param_grid={'C': [1, 10, 100, 1000]}, scoring='f1', verbose=1)

In [162]:
print(BestSVC.best_params_)
print(BestSVC.best_score_)

{'C': 1000}
0.831919274404921


In [158]:
y_50_BestSVCpred = BestSVC.predict(X_50_test)
print(confusion_matrix(y_50_test, y_50_BestSVCpred))
print(pd.DataFrame(classification_report(
    y_50_test, y_50_BestSVCpred, labels = None,
    target_names = ['no aki', 'aki'], sample_weight = None,
    digits = 2, output_dict = True)).round(2))

[[ 315  569]
 [ 509 2662]]
           no aki      aki  accuracy  macro avg  weighted avg
precision    0.38     0.82      0.73        0.6          0.73
recall       0.36     0.84      0.73        0.6          0.73
f1-score     0.37     0.83      0.73        0.6          0.73
support    884.00  3171.00      0.73     4055.0       4055.00
